# Demo CrewAi

In this Notebook we are going to demo how to use CrewAI with Azure OpenAI to create a restaurant recommendation assistant.

In this example we are going to use crewAI tools.

You can find more information about tools [here](https://github.com/crewAIInc/crewAI-tools)

1. **Install CrewAI**: First, we need to install the CrewAI package using pip.
2. **Configure Azure OpenAI Environment Variables**: Set up the necessary environment variables for Azure OpenAI.
3. **Import Python Modules**: Import the required Python modules for the demo.
4. **Define the Default LLM**: Set up the default language model using AzureChatOpenAI.
5. **Load Restaurant Categories**: Load the list of restaurant categories from a JSON file.
6. **Define CrewAI Tools**: Create tools for retrieving category IDs, addresses, and restaurant information.
7. **Create CrewAI Agent**: Define an agent with a specific role, goal, and tools.
8. **Create CrewAI Tasks**: Set up tasks for restaurant recommendations, category retrieval, and location finding.
9. **Instantiate Crew**: Create a Crew instance with the agent and tasks, and start the task execution.

Let get started with installing CrewAI!


## Questions

1. Where do I need to configure the tools? At the Agent, Task and or Crew?
2. Why do I need to configure category and location as input values for the Crew?
3. Why is keeps the crew running while the answer is given?

## References

- [I Created a CUSTOM AI Tool with CrewAI](https://www.google.com/search?q=crewai+tools+show+how+to+create+a+custom+tool&sca_esv=d29dcacd0903cf12&sxsrf=ADLYWIKoibXGp5VjDpt6pGktNyRBipnP3Q%3A1731753760079&ei=IHc4Z8q2BOupi-gP_eqFcA&ved=0ahUKEwjKxtS91eCJAxXr1AIHHX11AQ4Q4dUDCA8&uact=5&oq=crewai+tools+show+how+to+create+a+custom+tool&gs_lp=Egxnd3Mtd2l6LXNlcnAiLWNyZXdhaSB0b29scyBzaG93IGhvdyB0byBjcmVhdGUgYSBjdXN0b20gdG9vbDIFECEYoAEyBRAhGKABSIUzUNUMWIktcAF4AJABAJgBowGgAa0WqgEEMjguNbgBA8gBAPgBAZgCIaAC3RXCAgoQABiwAxjWBBhHwgIGEAAYFhgewgIIEAAYFhgKGB7CAgsQABiABBiGAxiKBcICCBAAGIAEGKIEwgIHECEYoAEYCsICBRAhGJIDwgIEECEYFZgDAIgGAZAGCJIHBDI4LjWgB7OPAQ&sclient=gws-wiz-serp#fpstate=ive&vld=cid:b4443448,vid:rcmMK-zkxrQ,st:142)
- [Tyler programmaming - Day04 Crew](https://github.com/tylerprogramming/ai/blob/main/crewai_series/day_04/src/day_04/config/agents.yaml)

In [100]:
%pip install -q crewai

Note: you may need to restart the kernel to use updated packages.


# Configure Azure OpenAI environment variables

In [101]:
import os

os.environ["AZURE_OPENAI_VERSION"] = "2023-03-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://fabricaio.openai.azure.com"
# Read the environment variable. 
azure_openai_key = os.getenv("AZURE_API_KEY")


print("Loaded environment variables for CrewAi demo")

Loaded environment variables for CrewAi demo


## Import Python Modules

Below code will imports the required Python Modules

In [102]:
from crewai import Agent, Task, Process, Crew
import os
from langchain_openai import AzureChatOpenAI
import json


# Define the default LLM

In [103]:
default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://fabricaio.openai.azure.com/"),
    api_key=azure_openai_key
)

# Load list of Azure Maps Restaurant Categories

Azure Maps uses restaurant categories and these are stored in a json file that needs to be loaded.

In [104]:
script_dir = 'C://Github//llmagents//crewainotebook'

# Construct the path to the JSON file
json_file_name = "restaurant_categories.json"
json_file_path = os.path.join(script_dir, json_file_name)
# Open and load the JSON file
with open(json_file_path, 'r') as file:
    restaurant_categories  = json.load(file)
    
print(restaurant_categories)

[{'category_id': '7315002', 'parent_id': '7315', 'category_name': 'African Restaurant', 'synonums': 'African Food,African Cuisine'}, {'category_id': '7315003', 'parent_id': '7315', 'category_name': 'American Restaurant', 'synonums': 'American Cuisine,American Food'}, {'category_id': '7315004', 'parent_id': '7315', 'category_name': 'Austrian Restaurant', 'synonums': 'Austrian Cuisine,Austrian Food'}, {'category_id': '7315005', 'parent_id': '7315', 'category_name': 'Barbecue Restaurant', 'synonums': 'Barbecue,Barbecue Food,BBQ,BBQ Restaurant,Barbecue Cuisine'}, {'category_id': '7315006', 'parent_id': '7315', 'category_name': 'Belgian Restaurant', 'synonums': 'Belgian Food,Belgian Cuisine'}, {'category_id': '7315007', 'parent_id': '7315', 'category_name': 'Bistro', 'synonums': None}, {'category_id': '7315008', 'parent_id': '7315', 'category_name': 'British Restaurant', 'synonums': 'British Cuisine,British Food'}, {'category_id': '7315009', 'parent_id': '7315', 'category_name': 'California


# Define CrewAI Tools

This section defines the following tools used in the CrewAI restaurant assistant demo:
- get_category_name: A function that retrieves the category name of a restaurant based on its ID or other identifying information.
- get_address: A function that fetches the address of a restaurant given its ID or other identifying information.
- get_restaurant_info: A function that gathers comprehensive information about a restaurant, including its name, address, category, and other relevant details.

These tools are essential for the functionality of the restaurant assistant, enabling it to provide accurate and detailed information about various restaurants.


In [105]:
from crewai_tools import BaseTool
import re
import http.client
import urllib.parse


# Define a tool that retrieves the category ID from the restaurant_categories.json file based on the category name
class GetCategoryIDTool(BaseTool):
    name: str = "Get Category ID Tool"
    description: str = (
        "Retrieves the category ID from the restaurant_categories.json file based on the category name stored in a variable called restaurant_categories ."
    )

    def _run(self, category: str) -> str:
        """
        Retrieves the category ID from the JSON file based on the category name.

        Args:
            category_name (str): The category name to search for.

        Returns:
            str: The category ID or an error message if not found.
        """
        for item in restaurant_categories:
            if re.search(category, item["category_name"], re.IGNORECASE):
                return item
        # If the category is not found, return an error message
        return "Category not found"


# Define geolocation_demo function
class GetAddressTool(BaseTool):
    name: str = "Get Address Tool"
    description: str = (
        "Retrieves the latitude and longitude of a city, e.g. Amsterdam, using the Azure Maps REST API."
    )

    def _run(self, query: str) -> str:
        """
        Retrieves the latitude and longitude based on the city name.

        Args:
            query (str): The place name to search for.

        Returns:
            str: The address of the place including the latitude and longitude in json format.
        """
        # Retrieve the subscription key from environment variables
        subscription_key = os.getenv("AzureMapsKey")

        if not subscription_key:
            raise ValueError(
                "Subscription key not found in environment variables. Please set 'AZURE_SUBSCRIPTION_KEY'."
            )

        conn = http.client.HTTPSConnection("atlas.microsoft.com")
        # Encode the query parameter to ensure it's URL-safe
        encoded_query = urllib.parse.quote(query)
        # Use the encoded query and subscription key in the URL
        conn.request(
            "GET",
            f"/search/address/json?api-version=1.0&query={encoded_query}&subscription-key={subscription_key}",
        )
        res = conn.getresponse()
        data = res.read()

        # Decode the response and load it as a JSON object
        json_data = json.loads(data.decode("utf-8"))

        # Pretty-print the JSON data
        return json.dumps(json_data, indent=4)
    
# Define function that retrieves restaurant information from Azure Maps REST API
# using lon and lat coordinates as parameters
# add the categorySet parameter to the URL to filter the search results by the restaurant category
class GetRestaurantInfo(BaseTool):
    name: str = "Get Restaurant Info Tool"
    description: str = (
        "Retrieves restaurant information based on the latitude and longitude coordinates using the Azure Maps REST API."
    )

    def _run(self, lat: str, lon: str, categorySet: str) -> str:
        """
        Retrieves restaurant information based on the latitude and longitude coordinates using the Azure Maps REST API.

        Args:
            lat (str): The latitude coordinate.
            lon (str): The longitude coordinate.
            categorySet (str): The category set to filter the search results.

        Returns:
            str: The restaurant information in json format.
        """
        # Retrieve the subscription key from environment variables
        subscription_key = os.getenv("AzureMapsKey")

        if not subscription_key:
            raise ValueError(
                "Subscription key not found in environment variables. Please set 'AZURE_SUBSCRIPTION_KEY'."
            )

        conn = http.client.HTTPSConnection("atlas.microsoft.com")
        # Use the latitude, longitude, and categorySet parameters in the URL
        conn.request(
            "GET",
            f"/search/fuzzy/json?api-version=1.0&query=restaurant&lat={lat}&lon={lon}&categorySet={categorySet}&subscription-key={subscription_key}",
        )
        res = conn.getresponse()
        data = res.read()

        # Decode the response and load it as a JSON object
        json_data = json.loads(data.decode("utf-8"))

        # Pretty-print the JSON data
        return json.dumps(json_data, indent=4)

# Create CrewAI Agent

Create a restaurant assistent Agent



In [112]:
restaurant_assistant = Agent(
  role='Restaurant Assistant',
  goal='Provide information about the restaurant recommendations based on the type of restaurant and their location',
  backstory='I am a restaurant assistant and I can help you with restaurant recommendations based on the type of restaurant and their location.',
  verbose=True,
  llm=default_llm,
  human_input_prompts=[
    "I am looking for a restaurant recommendation for a {category} restaurant in {location}.",
    "Can you recommend a {category} restaurant in {location}?",
    "I would like to know about the best {category} restaurant in {location}."
  ],
  tools=[GetCategoryIDTool(), GetAddressTool(), GetRestaurantInfo()] # Set of capabilities or functions that the agent can use to perform tasks. Expected to be instances of custom classes compatible with the agent’s execution environment. Tools are initialized with a default value of an empty list.
)

# Create CrewAI Task

Create a restaurant recommendation Task

In [114]:
restaurant_recommendation_task = Task(
  description='Assist with finding the best restaurants for customers based on their preferences in a specific location',
  agent=restaurant_assistant,  # Assigning the task to the restaurant assistant
  expected_output='A list of the top 5 restaurants for {category} food in the {location} that match the customer''s preferences',
  human_input= False,
  output_file='restaurant_assistant_output.md',
  verbose=True
)

# Create crewAI Task

Create a get category tool task

In [115]:
get_category_tool_task = Task(
  description='Retrieve the category ID from the JSON data based on the category name.',
  agent=restaurant_assistant,  # Assigning the task to the restaurant assistant
  expected_output='The category ID if found, or an error message.',
  tools=[GetCategoryIDTool()], # The functions or capabilities the agent can utilize to perform the task. Defaults to an empty list.
  human_input= False,
  verbose=True
)

# Create crewAI task

Create a task to find the location for which the user is asking a restaurant recommendation

In [116]:
get_restaurant_location_task = Task(
    description='Retrieve the location of the restaurant based on the user input.',
    agent=restaurant_assistant,
    expected_output='The location of the restaurant if found, or an error message.',
    tools=[GetAddressTool()], # The functions or capabilities the agent can utilize to perform the task. Defaults to an empty list.
    human_input=False,
    verbose=True
)

# Create crewAI task

Create a task to find all the restaurants for the location (lat, lon) and category.

In [117]:
get_restaurant_info_task = Task(
    description='Retrieve restaurant information based on the latitude and longitude coordinates.',
    agent=restaurant_assistant,
    expected_output='The restaurant information in json format.',
    tools=[GetRestaurantInfo()], # The functions or capabilities the agent can utilize to perform the task. Defaults to an empty list.
    human_input=False,
    verbose=True
)

# Instantiate crew

Now it's time to instantiate the crew and start the task execution

In [ ]:
# Create a Crew instance with the restaurant assistant agent and the tasks
# Why do I need to specify the inputs here? Cannot I just pass the following input? 
# "I'm hungry, I want a Mexican restaurant in Haarlem"
inputs = {
    "category": "Mexican",
    "location": "Halfweg, the Netherlands",
    "topic": "restaurant recommendation for a Mexican restaurant in Halfweg"
}

crew = Crew(
    agents=[restaurant_assistant],
    tasks=[restaurant_recommendation_task, get_category_tool_task, get_restaurant_location_task, get_restaurant_info_task],
    process=Process.sequential,
    inputs=inputs, # Why do I need to add this attribute here?
    verbose=True,
)

# Begin the task execution
crew.kickoff(inputs=inputs)


2024-11-18 16:23:08,857 - 31500 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


KeyError: 'category'